In [15]:
parent_dir = "./ctxt/"

In [16]:
import glob
text_files = glob.glob(parent_dir + '*.txt')
#print(text_files)

In [17]:
input_label_pairs = []
labels_count = 0
for text in text_files:
    if labels_count == 30:
        break
    label = text[-25:-20]
    if label.isdigit():
        label = int(label)
        #print(label)
        with open(text, 'r', encoding='utf-8-sig') as infile:
            program = infile.readlines()
            for i in range(len(program)):
                input_label_pairs.append((program[i],labels_count))
        labels_count += 1

In [18]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline, RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base',num_labels=labels_count)
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')


Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be 

In [19]:
print(labels_count)

30


In [20]:
print(input_label_pairs[0][0])
print(input_label_pairs[0][1])
print(input_label_pairs[-1][0])
print(input_label_pairs[-1][1])

#include<iostream> using namespace std;  int main(){ 	for (int i = 1; i <= 9; i++) 	{ 		for (size_t j = 1; j <= 9; j++) 		{ 			int ans = i * j; 			cout << i << "x" << j << "=" << ans << endl; 		} 	}     return 0; }

0
#include <bits/stdc++.h> using namespace std;  #define REP(i, n) for (int i = 0; i < (n); i++)  int main() {   int a[4], b[4], s = 0, t = 0, i;    REP(i, 4) cin >> a[i];   REP(i, 4) cin >> b[i];   REP(i, 4){     s += a[i];     t += b[i];   }   cout << max(s, t) << endl; }

29


In [21]:
print(len(input_label_pairs))

15000


In [22]:
import torch

In [23]:
from torch.utils.data import Dataset, DataLoader
class CPPDataset(Dataset):
    def __init__(self, cpp_label_pairs, tokenizer):
        self.tokenizer = tokenizer
        self.cpp_label_pairs = cpp_label_pairs

    def __len__(self):
        return len(self.cpp_label_pairs)

    def __getitem__(self, idx):
        cpp_label_pair = self.cpp_label_pairs[idx]
        input_code = self.tokenizer(cpp_label_pair[0], padding='max_length', max_length=512, truncation=True, return_tensors='pt')
        label = cpp_label_pair[1]
        return {"input_ids": input_code["input_ids"], "attention_mask": input_code["attention_mask"], "labels": label}

In [24]:
import torch
import torch.nn as nn

In [25]:
class Classifier(nn.Module):
    def __init__(self, num_classes,base_model):
        super().__init__()
        self.codebert = base_model
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.codebert(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        #print(outputs)
        return outputs

In [26]:
import tqdm

In [27]:
class CppTrainer:
    def __init__(self, model, tokenizer, train_dataset, val_dataset=None):
        torch.cuda.set_device(0)
        torch.cuda.empty_cache()
        self.model = Classifier(labels_count,model)
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Device name: {torch.cuda.get_device_name(0)}")
        print(f"Total memory: {torch.cuda.get_device_properties(self.device).total_memory / 1024**2:.2f} MB")
        print(f"Allocated memory: {torch.cuda.memory_allocated(self.device) / 1024**2:.2f} MB")
        print(f"Free memory: {torch.cuda.memory_reserved(self.device) / 1024**2:.2f} MB")

    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-5, weight_decay=0.01)
        criterion = nn.CrossEntropyLoss()

        train_loader = DataLoader(self.train_dataset, batch_size=8, shuffle=True)
        val_loader = DataLoader(self.val_dataset, batch_size=64) if self.val_dataset is not None else None

        for epoch in range(20):
            train_loss = 0.0
            self.model.train()
            c = 0
            for batch in train_loader:
                #print(batch)
                c += 1;
                if c == 2000:
                    print(train_loss)
                    c = 0
                    print("training")
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                optimizer.zero_grad()
                input_ids = input_ids.squeeze(1)
                #print(input_ids.shape)
                #print(labels.shape)
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                #print(loss)
                loss.backward()

                optimizer.step()

                train_loss += loss.item()
            if val_loader is not None:
                print("testing")
                #train_loss =10
                self.model.eval()
                val_loss = 0.0
                val_correct = 0
                with torch.no_grad():
                    for batch in val_loader:
                        input_ids = batch["input_ids"].to(device)
                        attention_mask = batch["attention_mask"].to(device)
                        labels = batch["labels"].to(device)
                        input_ids = input_ids.squeeze(1)
                        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                        _, predicted = torch.max(outputs.logits, dim=1)
                        #print(predicted)
                        #print(labels)
                        val_correct += (predicted == labels).sum().item()

                    print(f"Epoch {epoch+1}, Training Loss: {train_loss/len(self.val_dataset)}, Validation Accuracy: {val_correct/len(self.val_dataset)}, {val_correct}/{len(self.val_dataset)}")

            else:
                print(f"Epoch {epoch+1}, Training Loss: {train_loss/len(train_loader)}")
                


In [28]:
from sklearn.model_selection import train_test_split
# Split dataset into training and testing sets
train_data, test_data = train_test_split(input_label_pairs, test_size=0.2, random_state=42)

# Create datasets using the split data
train_dataset = CPPDataset(train_data, tokenizer)
test_dataset = CPPDataset(test_data, tokenizer)

In [29]:
trainer = CppTrainer(model, tokenizer, train_dataset,test_dataset)
trainer.train()


Device name: NVIDIA GeForce RTX 3070 Laptop GPU
Total memory: 8191.50 MB
Allocated memory: 0.00 MB
Free memory: 0.00 MB
testing
Epoch 1, Training Loss: 0.40885557454203564, Validation Accuracy: 0.98, 2940/3000
testing
Epoch 2, Training Loss: 0.050088213126175105, Validation Accuracy: 0.971, 2913/3000
testing
Epoch 3, Training Loss: 0.031098108352472384, Validation Accuracy: 0.9823333333333333, 2947/3000
testing
Epoch 4, Training Loss: 0.02316103195119649, Validation Accuracy: 0.982, 2946/3000
testing
Epoch 5, Training Loss: 0.024299758998754743, Validation Accuracy: 0.9786666666666667, 2936/3000
testing
Epoch 6, Training Loss: 0.016566405191241453, Validation Accuracy: 0.982, 2946/3000
testing
Epoch 7, Training Loss: 0.01686328394994295, Validation Accuracy: 0.9713333333333334, 2914/3000
testing
Epoch 8, Training Loss: 0.015414670802109565, Validation Accuracy: 0.976, 2928/3000
testing
Epoch 9, Training Loss: 0.014284718309800762, Validation Accuracy: 0.9816666666666667, 2945/3000
test

In [ ]:
torch.cuda.empty_cache()

In [15]:
dataset = CPPDataset(input_label_pairs, tokenizer)

In [ ]:
def generate_predictions(model, dataloader, tokenizer):
    predictions = []
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
        for output in outputs:
            prediction = tokenizer.decode(output, skip_special_tokens=True)
            predictions.append(prediction)
    return predictions

# Generate predictions on the test set and print some examples
dataset = CPPDataset(input_output_pairs, tokenizer)
predictions = generate_predictions(model, test_loader, tokenizer)
for i in range(10):
    print("Input:", test_pairs[i][0])
    print("Expected output:", test_pairs[i][1])
    print("Model output:", predictions[i])

In [19]:
dataset = CPPDataset(input_output_pairs, tokenizer)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [22]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)


In [24]:
trainer.train()

***** Running training *****
  Num examples = 100000
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 150000
  Number of trainable parameters = 124697433


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [10]:
from tqdm import tqdm, trange
inputs = []
outputs = []
for input_seq, output_seq in tqdm(input_output_pairs,total=1000000):
    inputs.append(tokenizer(input_seq, padding=True, truncation=True, return_tensors="pt"))
    outputs.append(tokenizer(output_seq, padding=True, truncation=True, return_tensors="pt"))


1024160it [23:06, 738.70it/s]                              


KeyboardInterrupt: 

In [12]:
print(len(inputs))
print(len(outputs))

1024161
1024160


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    logging_steps=1000,
    save_steps=5000,
    num_train_epochs=3,
    learning_rate=0.0001
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=inputs,
    eval_dataset=outputs
)

trainer.train()


D:\AN\envs\tf-gpu\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1024161
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1536243
  Number of trainable parameters = 124697433


RuntimeError: stack expects each tensor to be equal size, but got [1, 231] at entry 0 and [1, 115] at entry 1